<a href="https://colab.research.google.com/github/sharifashik591/Natural-language-processing/blob/main/Text_Generation_using_Tensorflow%2C_Keras_and_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Generating text using nlp and tensorflow

In [1]:
import tensorflow as tf
import string
import requests

In [2]:
data=requests.get('https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt')

In [3]:
data.text[:1000]

'This is the 100th Etext file presented by Project Gutenberg, and\nis presented in cooperation with World Library, Inc., from their\nLibrary of the Future and Shakespeare CDROMS.  Project Gutenberg\noften releases Etexts that are NOT placed in the Public Domain!!\n\nShakespeare\n\n*This Etext has certain copyright implications you should read!*\n\n<<THIS ELECTRONIC VERSION OF THE COMPLETE WORKS OF WILLIAM\nSHAKESPEARE IS COPYRIGHT 1990-1993 BY WORLD LIBRARY, INC., AND IS\nPROVIDED BY PROJECT GUTENBERG ETEXT OF ILLINOIS BENEDICTINE COLLEGE\nWITH PERMISSION.  ELECTRONIC AND MACHINE READABLE COPIES MAY BE\nDISTRIBUTED SO LONG AS SUCH COPIES (1) ARE FOR YOUR OR OTHERS\nPERSONAL USE ONLY, AND (2) ARE NOT DISTRIBUTED OR USED\nCOMMERCIALLY.  PROHIBITED COMMERCIAL DISTRIBUTION INCLUDES BY ANY\nSERVICE THAT CHARGES FOR DOWNLOAD TIME OR FOR MEMBERSHIP.>>\n\n*Project Gutenberg is proud to cooperate with The World Library*\nin the presentation of The Complete Works of William Shakespeare\nfor your

In [4]:
data=data.text.split('\n') #Split new line

In [5]:
# data[0:253]
data=data[253:]

In [6]:
len(data)

124204

In [7]:
data[0]

'  From fairest creatures we desire increase,'

In [8]:
data=" ".join(data)

In [9]:
data[:1000]

"  From fairest creatures we desire increase,   That thereby beauty's rose might never die,   But as the riper should by time decease,   His tender heir might bear his memory:   But thou contracted to thine own bright eyes,   Feed'st thy light's flame with self-substantial fuel,   Making a famine where abundance lies,   Thy self thy foe, to thy sweet self too cruel:   Thou that art now the world's fresh ornament,   And only herald to the gaudy spring,   Within thine own bud buriest thy content,   And tender churl mak'st waste in niggarding:     Pity the world, or else this glutton be,     To eat the world's due, by the grave and thee.                        2   When forty winters shall besiege thy brow,   And dig deep trenches in thy beauty's field,   Thy youth's proud livery so gazed on now,   Will be a tattered weed of small worth held:     Then being asked, where all thy beauty lies,   Where all the treasure of thy lusty days;   To say within thine own deep sunken eyes,   Were an al

In [10]:
def clean(doc):
  token=doc.split()
  table=str.maketrans('','',string.punctuation)
  token=[w.translate(table) for w in token]
  token=[word for word in token if word.isalpha()]
  token=[word.lower() for word in token]
  return token

In [11]:
tokens=clean(data)

In [12]:
tokens[0:20]

['from',
 'fairest',
 'creatures',
 'we',
 'desire',
 'increase',
 'that',
 'thereby',
 'beautys',
 'rose',
 'might',
 'never',
 'die',
 'but',
 'as',
 'the',
 'riper',
 'should',
 'by',
 'time']

In [13]:
len(tokens)

898199

In [14]:
len(set(tokens)) #total uniqie word

27956

In [15]:
#make sequence . 

length= 50+1
lines= []

for i in range(length, len(tokens)):
  seq=tokens[i-length:i]
  line=' '.join(seq)
  lines.append(line)

  if i>250000:
    break

print(len(lines))

249951


In [16]:
lines[10]

'might never die but as the riper should by time decease his tender heir might bear his memory but thou contracted to thine own bright eyes feedst thy lights flame with selfsubstantial fuel making a famine where abundance lies thy self thy foe to thy sweet self too cruel thou that'

In [17]:
tokens[500]

'were'

Build Model

In [18]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [19]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(lines)
sequence=tokenizer.texts_to_sequences(lines)

In [20]:
sequence=np.array(sequence)

x=sequence[:, :-1]
y=sequence[:,-1]

In [21]:
x[0] #first line

array([   48,  1595,  1516,    36,   487,  1683,    10,  2908,  1381,
        1224,   182,   121,   272,    21,    24,     1,  5584,    90,
          30,    92,  5583,    18,   843,  1183,   182,   207,    18,
         868,    21,    26,  3775,     5,   223,   119,  1325,   168,
       14869,    32,  2609,  2052,    16, 14868, 14867,   826,     6,
        3774,   101,  2222,   432,    32])

In [22]:
y[0]

365

In [23]:

vocab_size=len(tokenizer.word_index) +1

In [24]:
seq_length=x.shape[1]

In [ ]:
y=to_categorical(y, num_classes=voc_size)


LSTM Model

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])


In [ ]:
model.fit(x, y, batch_size = 256, epochs = 100)

In [ ]:
seed_text=lines[12343]

In [ ]:
def generate_text_seq(model, tokenizer, text_seq_length, seed_text, n_words):
  text = []

  for _ in range(n_words):
    encoded = tokenizer.texts_to_sequences([seed_text])[0]
    encoded = pad_sequences([encoded], maxlen = text_seq_length, truncating='pre')

    y_predict = model.predict_classes(encoded)

    predicted_word = ''
    for word, index in tokenizer.word_index.items():
      if index == y_predict:
        predicted_word = word
        break
    seed_text = seed_text + ' ' + predicted_word
    text.append(predicted_word)
  return ' '.join(text)

In [ ]:
print(generate_text_seq(model, tokenizer, seq_length, seed_text, 100))

'preposterously be stained to leave for them when thou art covetous and we pour their natural fortune grace the other fool as a monkey i cannot weep and ends you clown nay ay my lord ham aside the queen quoth thou into sport angelo to th capitol brutus patience peace night i am returnd to th field o th gout and a man that murdred of the greatest rout of her particular occasions that outruns the world side corioli did my performance gone cut the offender and take thee to you that dare not go and show me to be'

Tutorial Link : https://www.youtube.com/watch?v=VAMKuRAh2nc&list=PL-KyHBP_QcTvYB_9_lkd_PJQS_f7_Gm_e&index=1&t=1529s